In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import utilities


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# ticker = "AAPL"
# df = utilities.load_stock(ticker)
ticker = 'SPY'
df = utilities.load_index(ticker)

In [ ]:
# Add Indicators

df = utilities.RSI(df, 14)
df = utilities.RSI(df, 28)
df = utilities.RSI(df, 56)
df = utilities.RSI(df, 100)

df = utilities.MACD(df)

df = utilities.EMA(df, 10)
df = utilities.EMA(df, 50)
df = utilities.EMA(df, 200)

df = utilities.SMA(df, 10)
df = utilities.SMA(df, 50)
df = utilities.SMA(df, 100)

df = utilities.BollingerBands(df, 20)
df = utilities.BollingerBands(df, 50)
df = utilities.BollingerBands(df, 100)

df = utilities.ATR(df, 14)
df = utilities.ATR(df, 28)
df = utilities.ATR(df, 56)

df = utilities.OBV(df)

df = utilities.VWAP(df)

df = utilities.MFI(df, 14)
df = utilities.MFI(df, 28)
df = utilities.MFI(df, 56)

df = utilities.market_hours_only(df)
df = utilities.time_to_numeric(df)

In [ ]:
df.head()

In [ ]:
# map the "minute" column, from 570 to 930, to 0 to 1

df['minute'] = df['minute'].apply(lambda x: (x - 570) / (960 - 570))

In [ ]:
# Drop non-numeric columns
df = df.drop(columns=['timestamp', 'ticker', 'date', 'time'])

In [ ]:
df.head()

In [ ]:
# normalize all other columns
for col in df.columns:
    if col != 'enter':
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

In [ ]:
# Create a column called "value", which repserents how many of the next closes after this one are higher than the current close.
# The window is 390*5, to mean the next 5 days, so the last 100 values will be NaN.

days = 1

window = (390*days) - 1 # Adjusted the window size

# Create a column called "value"
df['value'] = df['close'].rolling(window=window).apply(lambda x: np.mean(x > x.iloc[-1])) 
df['value'] = df['value'].shift(-window)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# show the correlation matrix
correlation_matrix = df.corr()
plt.matshow(correlation_matrix)

plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90)
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar()
plt.gcf().set_size_inches(10, 10)

plt.show()

In [ ]:
filename = 'valued_{}.csv'.format(ticker)
df.to_csv(filename, index=False)